In [1]:
# graph functions
import networkx as nx
# library functions
import numpy as np
# data processing
import pandas as pd
# deepcopy
from copy import deepcopy
# visualization
from pyvis.network import Network

# Circuit analysis - overdetermined system of equations

Searching for amperage of each resistor in a circuit, using the Kirchhoff's laws.

## Table of contents

1. [Loading undirected weighted graph from file](#1-loading-undirected-weighted-graph-from-file)
2. [Utilities](#2-utilities)
   - [Finding cycles in a graph](#21-finding-cycles-in-a-graph)
3. [Solving the system of equations](#3-solving-the-system-of-equations)
   - [Building the system of equations](#31-building-the-system-of-equations)
   - [Building graph from equations](#32-building-graph-from-equations)
   - [Solving the system of equations](#33-solving-the-system-of-equations)
4. [Verifyng the system of equations](#4-verifyng-the-system-of-equations)
5. [Generating random circuits](#5-generating-random-circuits)
   - [Random resistance values](#51-random-resistance-values)
   - [Random graph (Erdos-Renyi)](#52-random-graph-erdos-renyi)
   - [3-regural graph](#53-3-regural-graph)
   - [Two random graphs with one bridge](#54-two-random-graphs-with-one-bridge)
   - [Grid graph](#55-grid-graph)
   - [Small world graph](#56-small-world-graph)
6. [Visualizing the circuit](#6-visualizing-the-circuit)

## 1. Loading undirected weighted graph from file

In [2]:
def load_graph(filename):
    graph = nx.Graph()        # circuit graph
    s, t = None, None         # source and target nodes
    E = None                  # electromotive force
    with open(filename) as f:
        try:
            s, t, E = map(int, f.readline().split())
        except:
            return error_message("ERROR1")
        for line in f:
            try:
                a, b, resistance = map(int, line.split())
                graph.add_edge(a, b, resistance=resistance)
            except:
                return error_message("ERROR2")
        if graph[s] == [] or graph[t] == []:
            return error_message("ERROR3")
    return (s, t, E), graph

def error_message(error_code):
    match error_code:
        case "ERROR1":
            print("ERROR: File first line format should be: <source node> <target node> <electromotive force>")
        case "ERROR2":
            print("ERROR: File format should be: <node1> <node2> <resistance>")
        case "ERROR3":
            print("ERROR: source or target node not in graph")
        case _:
            print("ERROR: unknown error")
    return error_code

## 2. Utilities

### 2.1 Finding cycles in a graph

In [3]:
def find_cycles(graph):
    return list(nx.cycle_basis(graph))

## 3. Solving the system of equations

### 3.1 Building the system of equations

In [4]:
def build_matrix(s, t, El, graph):
    if not (graph.has_edge(s, t) or graph.has_edge(t,s)):              # if there is no edge between s and t
            graph.add_edge(s, t)                                       # add a SEM edge
    V, E = len(graph.nodes), len(graph.edges)                          # number of nodes and edges
    edges_dict = {(min(edge), max(edge)): idx for idx, edge in enumerate(graph.edges)}
    cycles = find_cycles(graph)[:E]                                    # find all linearly independent cycles in the graph
    A = np.zeros((E, E))                                               # matrix of Kirchhoff's laws
    b = np.zeros(E)                                                    # electric current vector

    for i in range(len(cycles)):                                       # Kirchhoff's second law
        cycle_edges = zip(cycles[i], cycles[i][1:]+cycles[i][:1])
        for edge in cycle_edges:
            if edge == (s, t):
                b[i] = -El
            elif edge == (t, s):
                b[i] = El
            else:
                if edge in edges_dict:
                    A[i, edges_dict[edge]] = graph[edge[0]][edge[1]]["resistance"]
                else:
                    A[i, edges_dict[(edge[1], edge[0])]] = -graph[edge[0]][edge[1]]["resistance"]
            if edge == (cycles[i][0], cycles[i][1]):
                b[i] *= -1

    for i in range(E-len(cycles)):                                     # Kirchhoff's first law
        for j in graph[i]:                  
            if j > i:                        
                A[len(cycles)+i, edges_dict[(i,j)]] = 1 
            else:                            
                A[len(cycles)+i, edges_dict[(j,i)]] = -1
    
    if graph.has_edge(s, t):                                           # if there is an edge between s and t
        graph[s][t]["resistance"] = 0                                  # set its resistance to zero
    else:
        graph[t][s]["resistance"] = 0       
        
    return A, b, edges_dict

### 3.2 Building graph from equations

In [5]:
def build_circuit(graph, result, edges_dict):
    circuit = nx.DiGraph()                                           # directed graph of electric current
    for edge, idx in edges_dict.items():                             # add edges to the graph
        graph[edge[0]][edge[1]]["resistance"] = result[idx]          # set the resistance of the edge to the current
        if result[idx] > 0:                                          # if the current is positive
            circuit.add_edge(*edge, current=result[idx])             # add the edge to the graph
        else:                                                        # if the current is negative
            circuit.add_edge(edge[1], edge[0], current=-result[idx]) # switch the direction of the edge and add it to the graph
    return circuit

### 3.3 Solving the system of equations

In [6]:
def solve(graph_bundle):
    (s, t, E), graph = graph_bundle
    G_copy = deepcopy(graph)
    A, b, edges_dict = build_matrix(s, t, E, graph)
    result = np.linalg.solve(A, b)
    circuit = build_circuit(graph, result, edges_dict)
    return circuit

## 4. Verifyng the system of equations

In [7]:
def verify_graph(circuit, s, t, E, graph):

    # Kirchhoff's first law
    vertices_flow = [0 for _ in range(len(graph.nodes))]
    for edge in circuit.edges:       
        vertices_flow[edge[0]] -= circuit[edge[0]][edge[1]]["current"]
        vertices_flow[edge[1]] += circuit[edge[0]][edge[1]]["current"]
    for flow in vertices_flow:
        if abs(flow) > 1e-6:
            print("1", end=" ")
            return False
        
    # Kirchhoff's second law    
    cycles = nx.simple_cycles(circuit)
    circuit_edges = set(circuit.edges)
    for cycle in cycles:
        cycle_edges = zip(cycle, cycle[1:]+cycle[:1])
        cycle_flow = 0

        for edge in cycle_edges:
            if edge == (s, t):
                flow = E
            elif edge == (t, s):
                flow = -E
            elif edge in circuit_edges:
                flow = circuit[edge[0]][edge[1]]["current"] * graph[edge[0]][edge[1]]["resistance"]
            else:
                flow = circuit[edge[1]][edge[0]]["current"] * graph[edge[1]][edge[0]]["resistance"]
            if edge == (cycle[-1], cycle[1]):
                flow *= -1
            cycle_flow += flow
        
        if abs(cycle_flow) > 1e-6:
            print("2", end=" ")
            return False
        
    return True

## 5. Generating random circuits

### 5.1 Random resistance values

In [8]:
def random_resistance(graph):
    for v1, v2 in graph.edges:
        graph[v1][v2]["resistance"] = np.random.uniform(1, 10)
    return graph

### 5.2 Random graph (Erdos-Renyi)

In [9]:
def random_graph(v):
    return random_resistance(nx.gnp_random_graph(v, np.random.randint(v, v*(v-1)//2)))

### 5.3 3-regular graph (cubic)

In [10]:
def cubic_graph(v):
    return random_resistance(nx.random_regular_graph(3, v))

### 5.4 Two random graphs with one bridge

In [11]:
def bridge_graph(v):
    graph1 = random_graph(v//2)
    graph2 = random_graph(v - (v//2))
    connected = nx.disjoint_union(graph1, graph2)
    connected.add_edge(v//2-1, 1+v//2)
    return random_resistance(connected)

### 5.5 Grid graph

In [12]:
def grid_graph(v):
    grid = nx.grid_2d_graph(v, v)
    grid = nx.convert_node_labels_to_integers(grid, first_label=0, ordering='sorted')
    return random_resistance(grid)

### 5.6 Small-world graph

In [13]:
def small_world_graph(v):
    return random_resistance(nx.watts_strogatz_graph(v, 5, 0.2))

## 6. Visualizing the circuit

In [15]:
def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(*rgb)

def get_edge_color(value, max_value):
    low = (127, 255, 0) # light green
    high = (100, 11, 11) # dark red
    factor = abs(value / max_value)
    interpolated_rgb = tuple(int(part[0] * (1 - factor) + part[1] * factor) for part in zip(low, high))
    return rgb_to_hex(interpolated_rgb)

def show_graph(circuit, graph, title):
    BLACK = "000000"
    V = len(graph.nodes)
    g = Network(directed=True)

    g.add_nodes([i for i in range(V)],
                title=["Node " + str(i) for i in range(V)],
                size=[5 for i in range(V)], color=[BLACK for i in range(V)])

    max_current = max([circuit[a][b]["current"] for a, b in circuit.edges])
    for a, b in graph.edges:
        if graph[a][b]["resistance"] == 0: 
            label = f"U={10}V"
        else: 
            label = f"R={graph[a][b]['resistance']:.2f}Ω"
        
        if (a, b) in circuit.edges:
            label += f", I={circuit[a][b]['current']:.2f}A"
            color = get_edge_color(circuit[a][b]["current"], max_current)
        else:
            label += f", I={circuit[b][a]['current']:.2f}A"
            color = get_edge_color(circuit[b][a]["current"], max_current)
        
        g.add_edge(a, b, title=label, label=label, color=color)

    g.show(f"{title}.html")

In [16]:
types = [random_graph, cubic_graph, bridge_graph, grid_graph, small_world_graph]
for v in [10, 20]:
    for t in types:
        print(t.__name__, end= " ")
        if t == grid_graph: graph = t(5)
        else: graph = t(v)
        circuit = solve(((0, v-1, 10), graph.copy()))
        print(verify_graph(circuit, 0, v-1, 10, graph))
        if v==10: show_graph(circuit, graph, t.__name__ + "10")

print("done")


random_graph True
cubic_graph True
bridge_graph True
grid_graph True
small_world_graph True
random_graph True
cubic_graph True
bridge_graph True
grid_graph True
small_world_graph True
done


------------------------------------------------